In [2]:
!pip install optuna -q

     |████████████████████████████████| 184kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 36.8MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.0MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 8.9MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 46.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 5.9MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 7.7MB/s  eta 0:00:01


In [92]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

import optuna

In [93]:
np.random.seed(42)
random.seed(42)

In [94]:

X_test_ = pd.read_csv('../data/Xte.csv',sep=',',index_col=0)
X_train_ = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0)

X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
X_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0)

In [95]:
y['Bound'] = y.Bound.apply(lambda x: -1 if x == 0 else 1)
y.head()
y = y.Bound.values
y

array([ 1, -1,  1, ...,  1,  1,  1])

In [96]:
# print('x_train: {} y_train {}'.format(X_preprocess[:2000,:].shape,y.shape))
# print('test: {}'.format(X_preprocess[2000:,:].shape))

In [97]:

def get_train_test(X,y,p):
    X = scale(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=p, random_state=42)
    print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

# Model Test

In [98]:
def rbf_kernel_element_wise(x, y, sigma=1):
    K =  np.exp(-np.sum((x-y)**2)/(2*sigma**2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(X1, X2, power=2):
    return np.power((1 + linear_kernel(X1, X2)),power)


def LevenshteinDistance(str1,str2):
    '''
    Compute the edit distance between str1 and str2
    Param: @(str1): (str) string 1 for the comparison
    @(str2): (str) string 2 for the comparison
    Return (int) distance
    '''
    len_s1 = len(str1) +1
    len_s2 = len(str2) +1
    m = np.zeros((len_s1,len_s2))
    for i in range(len_s1):
        m[i,0] = i
    
    for j in range(len_s2):
        m[0,j] = j
    
    for i in range(1,len_s1):
        for j in range(1,len_s2):
            if str1[i-1]==str2[j-1]:
                m[i,j]= min(m[i-1,j]+1,m[i,j-1]+1,m[i-1,j-1])
            else:
                m[i,j] =min(m[i-1,j]+1,m[i,j-1]+1,m[i-1,j-1]+1)
    return m[-1,-1]


def rbf_kernel(X1, X2, sigma=10):
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def linear_kernel(X1, X2):
    return X1.dot(X2.T)

In [99]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel,
        'gaussian':gaussian_kernel
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf' or self.kernel_name == 'gaussian':
            params['sigma'] = kwargs.get('sigma', None)
        if self.kernel_name == 'polynomial':
            params['power'] = kwargs.get('power', None)
        return params

    def fit(self, X, y, **kwargs):
        return self
        
    def decision_function(self, X):
        pass

    def predict(self, X):
        pass

In [100]:
class KernelRidgeRegression(KernelMethodBase):
    '''
    Kernel Ridge Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelRidgeRegression, self).__init__(**kwargs)

    def fit(self, X, y, sample_weights=None):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        self.y_train = y
        
        if sample_weights is not None:
            w_sqrt = np.sqrt(sample_weights)
            self.X_train = self.X_train * w_sqrt[:, None]
            self.y_train = self.y_train * w_sqrt
        
        A = self.kernel_function_(X,X,**self.kernel_parameters)
        A[np.diag_indices_from(A)] = np.add(A[np.diag_indices_from(A)],n*self.lambd)
        # self.alpha = (K + n lambda I)^-1 y
        self.alpha = np.linalg.solve(A , self.y_train)

        return self
    
    def decision_function(self, X):
        K_x = self.kernel_function_(X,self.X_train, **self.kernel_parameters)
        return K_x.dot(self.alpha)
    
    def predict(self, X):
        return self.decision_function(X)

In [101]:
def cross_validate(x_data,y_data,kernel=None,lambd=0.2,sigma=0.5,k=5,power=2):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data.reshape(-1,1),k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
            
        model = KernelRidgeRegression(
                kernel=kernel,
                lambd=lambd,
                sigma=sigma,
                power=power
            ).fit(x_train, y_train)
        result = sum(np.sign(model.predict(x_test))==y_test)/len(y_test)
        aggrigate_result.append(result)
        
        value = sum(aggrigate_result)/len(aggrigate_result)
    return value

In [102]:
train_data = pd.concat([X_train_ , pd.DataFrame(y)],axis=1)

def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]



In [44]:
from sklearn.feature_extraction.text import CountVectorizer

train_data['words'] = train_data.seq.apply(lambda x: ' '.join(getKmers(x)))
X_test_['words'] = X_test_.seq.apply(lambda x: ' '.join(getKmers(x)))
train_data.shape

data = pd.DataFrame(pd.concat([train_data.words,X_test_.words],axis=0))
train_text = data.words.values

cv = CountVectorizer(ngram_range=(2,2),max_features=1500,min_df=10,binary=True)
X = cv.fit_transform(train_text)
X = X.todense()


X.shape

,Bound
Id,
0,1
1,0
2,1
3,0
4,1
...,...
1995,1
1996,0
1997,1


In [45]:
cross_validate(np.array(X)[:2000,:],y,kernel='polynomial',lambd=0.001,k=4,sigma=0.2,power=5)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [133]:
def objective(trial):
    lambd = trial.suggest_loguniform('lambd', 1e-7, 3)
    sigma = trial.suggest_loguniform('sigma', 1e-7, 3)
    k =  trial.suggest_categorical('k', [4,5,8,10])
    power =  trial.suggest_int('power', 2,15)
    kernel =  trial.suggest_categorical('kernel', ['linear','rbf','polynomial'])
    
    return cross_validate(np.array(X)[:2000,:],y,kernel=kernel,lambd=lambd,k=4,sigma=sigma,power=power)


# cross_validate(X_train_mat100, y,lamda=0.01,k=4)
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
df = study.optimize(func=objective, n_trials=1000,show_progress_bar=True)


df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.sort_values(by=['value'])

In [103]:
def base2int(c):
    return {'a':0,'c':1,'g':2,'t':3}.get(c,0)

def index(kmer):
    base_idx = np.array([base2int(base) for base in kmer])
    multiplier = 4** np.arange(len(kmer))
    kmer_idx = multiplier.dot(base_idx)
    return kmer_idx
    
    
def spectral_embedding(sequence,kmer_size=3):
    kmers = getKmers(sequence,kmer_size)
    kmer_idxs = [index(kmer) for kmer in kmers]
    one_hot_vector = np.zeros(4**kmer_size)
    
    for kmer_idx in kmer_idxs:
        one_hot_vector[kmer_idx] += 1
    return one_hot_vector




data = pd.DataFrame(pd.concat([X_train_.seq,X_test_.seq],axis=0))

train_text = data.seq.values
# X_train_['kmers'] = X_train_.seq.apply(lambda x:list(spectral_embedding(x,kmer_size=3)))

kmer_data = []
for i in train_text:
    kmer_data.append(spectral_embedding(i,kmer_size=8))
    
np.array(kmer_data).shape

(3000, 65536)

In [104]:
y

array([ 1, -1,  1, ...,  1,  1,  1])

In [105]:
def objective(trial):
    lambd = trial.suggest_loguniform('lambd', 1e-9, 3)
    sigma = trial.suggest_loguniform('sigma', 1e-9, 3)
    k =  trial.suggest_categorical('k', [4,5,8])
    power =  trial.suggest_int('power', 2,15)
    kernel =  trial.suggest_categorical('kernel', ['linear','rbf','polynomial'])
    
    return cross_validate(np.array(kmer_data)[:2000,:],y,kernel=kernel,lambd=lambd,k=4,sigma=sigma,power=power)


# cross_validate(X_train_mat100, y,lamda=0.01,k=4)
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
df = study.optimize(func=objective, n_trials=500,show_progress_bar=True)




/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2020-05-29 01:42:30,723] Finished trial#0 with value: 0.617 with parameters: {'lambd': 0.06702378064927854, 'sigma': 0.0008445163408804387, 'k': 5, 'power': 15, 'kernel': 'polynomial'}. Best is trial#0 with value: 0.617.
[I 2020-05-29 01:42:35,385] Finished trial#1 with value: 0.6345000000000001 with parameters: {'lambd': 0.02085639648275362, 'sigma': 3.9040236143315835e-07, 'k': 4, 'power': 3, 'kernel': 'polynomial'}. Best is trial#1 with value: 0.6345000000000001.
[I 2020-05-29 01:42:42,199] Finished trial#2 with value: 0.0 with parameters: {'lambd': 0.0002530426405211842, 'sigma': 2.3341904470467587e-08, 'k': 4, 'power': 15, 'kernel': 'rbf'}. Best is trial#1 with value: 0.6345000000000001.
[I 2020-05-29 01:42:46,837] Finished trial#3 with value: 0.6275000000000001 with parameters: {'lambd': 0.034721519700855574, 'sigma': 3.570704065683582e-05, 'k': 8, 'power': 5, 'kernel': 'polynomial'}. Best is trial#1 with value: 0.6345000000000001.
[I 2020-05-29 01:42:51,401] Finished trial#4 

In [106]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.sort_values(by=['value'])

,number,value,duration,params_k,params_kernel,params_lambd,params_power,params_sigma
2,2,0.0000,00:00:06.809197,4,rbf,2.530426e-04,15,2.334190e-08
47,47,0.0000,00:00:06.865268,8,rbf,1.205098e+00,12,1.104133e-05
5,5,0.0000,00:00:06.875649,8,rbf,6.856272e-04,8,1.349338e-06
36,36,0.0000,00:00:06.991329,8,rbf,7.404650e-02,14,1.012862e-07
20,20,0.0000,00:00:06.789282,4,rbf,4.958577e-06,8,1.730696e-04
16,16,0.0000,00:00:06.922304,4,rbf,2.183853e-08,9,8.159557e-03
0,0,0.6170,00:00:04.664926,5,polynomial,6.702378e-02,15,8.445163e-04
38,38,0.6170,00:00:04.673783,8,polynomial,7.108641e-02,15,7.696706e-06
34,34,0.6170,00:00:04.793176,8,polynomial,1.939239e-01,15,6.580585e-08
8,8,0.6190,00:00:04.715475,4,polynomial,4.728263e-09,7,4.085384e-05


In [107]:
X_train, X_test, y_train, y_test = get_train_test(np.array(kmer_data)[:2000,:],y,0.1)

(1800, 65536) (200, 65536) (1800,) (200,)


In [110]:
model = KernelRidgeRegression(
                kernel='linear',
                lambd=2.111926e-01,
                sigma=8.625947e-08,
                power=15
            ).fit(X_train, y_train)
result = sum(np.sign(model.predict(X_test))==y_test)/len(y_test)
result

0.655

In [113]:
cross_validate(np.array(kmer_data)[:2000,:],y,kernel='linear',k=5,power=15,lambd=2.111926e-01,sigma=8.625947e-08)

array([0.6485])

In [119]:
X_test_final = np.array(kmer_data)[2000:,:]


sumbission = []
for i in range(len(X_test_final)):
    r1 = np.sign(model.predict(X_test_final[i]))
    
    if r1 == 1:
        sumbission.append([i,int(r1)])
    elif r1 == -1:
        sumbission.append([i,0])
    else:
        print('problem')
        
    
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('cv_65_linear_overfitted.csv',index=False)

df.head(15)

,Id,Bound
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,0
8,8,0
9,9,0


In [118]:
df.shape

(1000, 2)